In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import seaborn as sns
from matplotlib import pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
plt.style.use("seaborn-whitegrid")
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import (OrdinalEncoder, StandardScaler, 
                                   MinMaxScaler, PolynomialFeatures,
                                   PowerTransformer)

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import Lasso, RidgeCV
from sklearn.ensemble import (RandomForestRegressor, AdaBoostRegressor,
                             GradientBoostingRegressor, ExtraTreesRegressor)
from sklearn.ensemble import VotingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.svm import SVC 
from xgboost import XGBRegressor,XGBClassifier
%matplotlib inline

from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV


In [2]:
def get_csv_data(name : str):
    return pd.read_csv(name)

In [4]:
input1='../input/jobathon-august-2022/test_Bk2wfZ3.csv'
input2='../input/jobathon-august-2022/sample_submission_LJ2N3ZQ.csv'

In [6]:
sub_s=get_csv_data(input2)
sub_s

In [5]:
test=get_csv_data(input1)
test

In [3]:
input='../input/jobathon-august-2022/train_F3fUq2S.csv'
train=get_csv_data(input)
train

In [7]:
#check null values
train.isnull().sum()

In [8]:
num_cols=[ x for x in train.columns if train[x].dtype in ['int','float']]
num_cols

In [12]:
num_cols1=[ x for x in test.columns if test[x].dtype in ['int','float']]
num_cols1

In [9]:
cat_cols=[ x for x in train.columns if train[x].dtype not in ['int','float']]
cat_cols

In [11]:
fig=plt.figure(figsize=(15,14))
for i, f in enumerate(num_cols):
    # New subplot
    plt.style.use('ggplot')
    plt.subplot(6,5,i+1)
    sns.histplot(x=train[f])
    plt.title(f'Feature: {f}')
    plt.xlabel('')
    
fig.suptitle('Feature distributions',  size=20)
fig.tight_layout()  
plt.show()

In [13]:
fig=plt.figure(figsize=(15,14))
for i, f in enumerate(num_cols1):
    # New subplot
    plt.style.use('ggplot')
    plt.subplot(6,5,i+1)
    sns.histplot(x=test[f])
    plt.title(f'Feature: {f}')
    plt.xlabel('')
    
fig.suptitle('Feature distributions',  size=20)
fig.tight_layout()  
plt.show()

In [14]:
perc =[.50, .90]
subsets = pd.IndexSlice[['90%'], 'click_rate']
train.describe(percentiles = perc).style.applymap(lambda x: "background-color: magenta", subset=subsets)

In [15]:
train['times_of_day']=train['times_of_day'].astype('category').cat.codes
test['times_of_day']=test['times_of_day'].astype('category').cat.codes
test

In [16]:
feat=[x for x in train.columns if x not in ['campaign_id','click_rate']]
feat

In [18]:
y=train.click_rate
X=train[feat]
test1=test[feat]
test1


In [23]:
def train1(model):
    
    clf = Pipeline(steps=[
                       ('model', model)
                     ])

    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    ypred = 0
    total_loss = 0
    for train_indx, test_indx in cv.split(X):
        X_train, X_val = X.iloc[train_indx], X.iloc[test_indx]
        y_train, y_val = y.iloc[train_indx], y.iloc[test_indx]
        clf.fit(X_train, y_train)
        
        yhat = clf.predict(X_val)
        #score =mean_squared_error(yhat, y_val,squared=False)
        score=r2_score(yhat, y_val)
        print(f"Loss:{score}")
        ypred += clf.predict(test1) / 5
        total_loss += score / 5
        
    print(f"Avg. Loss: {total_loss}")     
    return ypred


In [32]:
name = 'AV1'
def greet():
    name = 'AV2'
    def hello():
        name = 'AV3'
        print("Hello "+name)
    hello()
    print("Hello "+name)
greet()
print("Hello "+name)

In [34]:
model =LGBMRegressor(n_estimators=400)#RandomForestRegressor()#CatBoostRegressor(n_estimators=1500)#GradientBoostingRegressor()#XGBRegressor()#LGBMRegressor(n_estimators=150)#ExtraTreesRegressor()#RandomForestRegressor()#LGBMRegressor(n_estimators=150)#CatBoostRegressor()#RandomForestRegressor()LGBMRegressor(n_estimators=300)
#model=Lasso()
final_prediction = train1(model)


In [35]:
test['click_rate']=final_prediction
df2=test[['campaign_id','click_rate']]
df2.to_csv('final_sub.csv',index=False)
df2